https://minimin2.tistory.com/137

https://leo-bb.tistory.com/62?category=858291

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html

# 결측치 많은 변수들

'user_test_correct_cnt_per_test': 372603,

'user_acc_per_test': 372603,

'user_total_acc': 365164,

 'user_total_correct_cnt': 365164

In [1]:
!pip install pycaret[full] ngboost shap


  Using cached shap-0.39.0.tar.gz (356 kB)
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xux5s9ep/shap/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xux5s9ep/shap/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-bq5pmchi
       cwd: /tmp/pip-install-xux5s9ep/shap/
  Complete output (119 lines):
  The nvcc binary could not be located in your $PATH. Either  add it to your path, or set $CUDAHOME to enable CUDA
  Error building cuda module: TypeError('cannot unpack non-iterable NoneType object')
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/shap
  copying shap/__init__.py -> build/lib.linux-x86_64-3.7/shap
  copying shap/_

# LGBM with pycaret



In [1]:
import numpy as np
import pandas as pd
import os
import random
import pickle
from pycaret.classification import *
from pycaret.utils import check_metric
import time
from datetime import timedelta, timezone, datetime
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm_notebook, tqdm
import json
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 데이터 가져오기

In [2]:
upper_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset'
train_path = f'{upper_dir}/train_data.csv'
# train_path = f'{upper_dir}/new_train_data.csv'
test_path = f'{upper_dir}/test_data.csv'
submission_path = f'{upper_dir}/sample_submission.csv'

In [3]:
train = pd.read_csv(train_path) 
test = pd.read_csv(test_path)

answerCode2bool = {'userID':object,  'answerCode': 'int16', 'KnowledgeTag':object}
train = train.astype(answerCode2bool)
test = test.astype(answerCode2bool)
# train.Timestamp = pd.to_datetime(train.Timestamp)
# test.Timestamp = pd.to_datetime(test.Timestamp)

NameError: name 'pd' is not defined

In [ ]:
# all_data = True
# if all_data == True:
#     train = pd.concat([train,test],axis=0)
#     train = train.sort_values(['userID'])
#     train.index = range(train.shape[0])
print(train.shape)

(2266586, 6)


# Feature Engineering

## Train에서 미리 추출해야할 변수들

In [5]:
# # trian에서 각 문제별 통계량 추출
# # 아래의 피쳐들은 train에서 뽑은 값들을 test에 대입해주어야함.

# testId_mean_sum_count = train.groupby(['testId'])['answerCode'].agg(['mean','sum','count']).to_dict()
# assessmentItemID_mean_sum_count = train.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
# KnowledgeTag_mean_sum_count = train.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','count']).to_dict()

total_num_prob_in_test = train[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()  
testid_maxlen_dict = total_num_prob_in_test.to_dict()

# LB Prediction해야하는 문항 ID들을 파악
set_assessmentItemID = set(test.loc[test.answerCode == -1, 'assessmentItemID'].values)
set_testID = set(test.loc[test.answerCode == -1, 'testId'].values)

## Input Data에 대해서 만들어줘야 하는 피쳐들

In [6]:
# def jinhyun_feature(df):
#     df.index = range(df.shape[0])
#     whole_count = df.groupby('assessmentItemID').size()
#     correct_count = df.groupby('assessmentItemID').answerCode.sum()
#     wrong_count =  whole_count - correct_count
#     df['time_elapsed_over_300'] = 0
#     df.loc[list(df[df.problem_time_diff_in_user >= 300].index), 'time_elapsed_over_300'] = 1
#     over_count = df.groupby('assessmentItemID').time_elapsed_over_300.sum()
#     level_assessment = (correct_count - wrong_count) / (whole_count - over_count)* 100
#     level_assessment_2_dict = level_assessment.to_dict()
#     df['level_assessment'] = df['assessmentItemID'].map(level_assessment_2_dict)
#     return df


In [7]:
def minyong_feature(df):
    df['test_pre'] = df.testId.map(lambda x: int(x[1:4]))
    df['test_post'] = df.testId.map(lambda x: int(x[-3:]))
    return df

In [8]:
def ara_featunre(df):
    # ------- user의 평균 문제 풀이 시간 (시험지 별 체크)
    next_df = df.shift(-1).fillna(-1)
    prev_df = df.shift(1).fillna(-1)
    df["diff"] = pd.to_datetime(next_df["Timestamp"]) - pd.to_datetime(df["Timestamp"])

    pr_user_id = df["userID"][0]
    pr_test_num, diff = 1, 0
    user_current_test = defaultdict(int)
    user_ox_count = dict()
    user_continue_ox_count = dict()
    user_assessmentnum_answerrate = dict()

    user_assessment_avg, user_test_done_time = [], []
    user_answer_cnt, user_wrong_cnt = [], []
    user_continue_answer_cnt, user_continue_wrong_cnt = [], []
    user_test_answer_rate = []
    i=-1
    for user_id, pr_test, pr_assess, nx_test, nx_assess, diff_time, prev_answer in tqdm(zip(df["userID"], df["testId"], df["assessmentItemID"], next_df["testId"], next_df["assessmentItemID"], df["diff"], prev_df["answerCode"])):
        i+=1

        if user_id != pr_user_id: # 다른 user 시작
            pr_test_num, diff = 1, 0
            user_current_test = defaultdict(int)
            user_ox_count = dict()
            user_continue_ox_count = dict()
            user_assessmentnum_answerrate = dict()
            pr_user_id = user_id
            prev_answer = -1
        
        if user_current_test[pr_test] == 0 or nx_test == -1: # 새로운 시험지거나 맨 마지막일 경우
            prev_answer = -1
        
        if pr_test not in user_current_test:
            user_current_test[pr_test] = 1 # 현재 시험지의 푼 문항 수 체크
        else:
            user_current_test[pr_test] += 1
        
        if pr_test not in user_ox_count:
            prev_count = [1, 0] if prev_answer == 1 else [0, 1] # Answer / Wrong count
            prev_num_rate = [1, 1, 1.0] if prev_answer == 1 else [1, 0, 0.0] # 푼 문제 수, 현재까지 정답 수, 정답률
            if prev_answer == -1:
                prev_count = [0, 0]
            
            user_ox_count[pr_test] = deepcopy(prev_count)
            user_continue_ox_count[pr_test] = deepcopy(prev_count)
            user_assessmentnum_answerrate[pr_test] = deepcopy(prev_num_rate) # 시험지 별 푼 문항 수, 정답률
        else:
            if prev_answer == 1:
                user_ox_count[pr_test][0] += 1
                user_continue_ox_count[pr_test][0] += 1 # 이전 답이 1 이라면 +1
                user_continue_ox_count[pr_test][1] = 0  # 연속 오답 count는 초기화
                user_assessmentnum_answerrate[pr_test][0] += 1 # 푼 문항 수 증가
                user_assessmentnum_answerrate[pr_test][1] += 1 # 푼 문항 중 정답수 증가
        
            else:
                user_ox_count[pr_test][1] += 1
                user_continue_ox_count[pr_test][0] = 0  # 연속정답 count는 초기화
                user_continue_ox_count[pr_test][1] += 1 # 이전 답이 0 이라면 +1
                user_assessmentnum_answerrate[pr_test][0] += 1 # 푼 문항 수 증가
        
        user_assessmentnum_answerrate[pr_test][2] = user_assessmentnum_answerrate[pr_test][1]/user_assessmentnum_answerrate[pr_test][0] # 정답률 갱신

        user_answer_cnt.append(user_ox_count[pr_test][0])
        user_wrong_cnt.append(user_ox_count[pr_test][1])

        user_continue_answer_cnt.append(user_continue_ox_count[pr_test][0])
        user_continue_wrong_cnt.append(user_continue_ox_count[pr_test][1])

        user_test_answer_rate.append(user_assessmentnum_answerrate[pr_test][2])


        if user_current_test[pr_test] == testid_maxlen_dict[pr_test]: # 시험지의 문제를 모두 다 풀었다면
            del user_current_test[pr_test] # 풀고있는 시험지에서 제외 (모두 다 풀었음)
            del user_ox_count[pr_test]
            del user_continue_ox_count[pr_test]
            del user_assessmentnum_answerrate[pr_test]

        if len(user_current_test) == 0 or nx_test == -1: # 만약 usre가 시작한 시험지를 모두 푸는데 완료했다면
            user_test_done_time.append(diff + user_assessment_avg[-1]) # 마지막 문제는 현재 걸린 시간에 평균적으로 걸린 시간만큼만 더해줌
            user_assessment_avg.append(user_assessment_avg[-1]) # 마지막 문제는 이전의 평균 값으로 사용

            pr_test_num, diff = 1, 0
            user_current_test = defaultdict(int)
            user_ox_count = dict()
            user_continue_ox_count = dict()
            user_assessmentnum_answerrate = dict()

        else: # 하나라도 끝내지 못한 시험지가 남아있다면
            diff += diff_time.total_seconds()
            user_test_done_time.append(diff)
            user_assessment_avg.append(diff/pr_test_num)
        
        pr_test_num += 1
    
    df = pd_join(df, "userSolTime", user_assessment_avg) # user의 문항을 푸는데 마친 평균 시간
    df = pd_join(df, "userTestTime", user_test_done_time) # user의 시험지 별 푸는데 마친 시간
    df = pd_join(df, "userTestAnswer", user_answer_cnt) # user의 시험지 별 정답 수
    df = pd_join(df, "userTestWrong", user_wrong_cnt) # user의 시험지 별 오답 수
    df = pd_join(df, "userTestContAnswer", user_continue_answer_cnt) # user의 시험지 별 연속 정답 수
    df = pd_join(df, "userTestContWrong", user_continue_wrong_cnt) # user의 시험지 별 연속 오답 수
    df = pd_join(df, "userTestCorrectRate", user_test_answer_rate) # user의 시험지 별 정답률

    # 유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['correctAnswer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['correctAnswer'] = df['correctAnswer'].fillna(0)
    df['totalAnswer'] = df.groupby('userID')['answerCode'].cumcount()
    df['userAcc'] = df['correctAnswer']/df['totalAnswer']
    df['userAcc'] = df['userAcc'].fillna(0)
    return df

# 결측치 채우기

In [9]:
# 아라 feature 2개 결측치 채우기
def ara_fillna(df):
    df['userAcc'] = df['userAcc'].fillna(0)
    df['correctAnswer'] = df['correctAnswer'].fillna(0)
    return df

# 내꺼 feature 4개 결측치 채우기
def mjh_fillna(df):
    df['user_total_correct_cnt'] = df['user_total_correct_cnt'].fillna(0)
    df['user_total_acc'] = df['user_total_acc'].fillna(0)
    df['user_test_correct_cnt_per_test'] = df['user_test_correct_cnt_per_test'].fillna(0)
    df['user_acc_per_test'] = df['user_acc_per_test'].fillna(0)
    return df

In [10]:
asset_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset/dict_for_feature'
# LB prediction에서 맞춰야하는 문항 ID 파악
def pd_join(data, column_name, value):
    value = pd.DataFrame(value, columns=[f"{column_name}"])
    data = data.join(value)
    return data
def save_pickle(path, values):
    os.makedirs(asset_dir, exist_ok=True)
    with open(f"{asset_dir}/{path}", 'wb') as f:
        pickle.dump(values, f)

def load_pickle(path):
    with open(f"{asset_dir}/{path}", 'rb') as f:
        load_values = pickle.load(f)
    return load_values


# Feature Engineering

In [11]:
minyoung = True
jinhyun = True
ara = True
ara_fillna_bool = True

mjh_fillna_bool = True

# def mjh_mean_fillna(df):
#     df['user_total_correct_cnt'] = df['user_total_correct_cnt'].fillna(df['user_total_correct_cnt'].mean())
#     df['user_total_acc'] = df['user_total_acc'].fillna(df['user_total_acc'].mean())
#     df['user_test_correct_cnt_per_test'] = df['user_test_correct_cnt_per_test'].fillna(df['user_test_correct_cnt_per_test'].mean())
#     df['user_acc_per_test'] = df['user_acc_per_test'].fillna(df['user_acc_per_test'].mean())
#     return df
# df = mjh_mean_fillna(df)
# df_test = mjh_mean_fillna(df_test)

In [12]:
def feature_engineering(df, is_train):
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    
    if is_train:
        # 본 피쳐들은 train에서 얻어진 값을 그대로 유지합니다.
        testId_mean_sum_count = df.groupby(['testId'])['answerCode'].agg(['mean','sum','count']).to_dict()
        save_pickle("testId_mean_sum_count.pk",testId_mean_sum_count)

        assessmentItemID_mean_sum_count = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("assessmentItemID_mean_sum_count.pk",assessmentItemID_mean_sum_count)

        KnowledgeTag_mean_sum_count = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("KnowledgeTag_mean_sum_count.pk",KnowledgeTag_mean_sum_count)


    # -> inference할 때는, 저장된 정답률을 가져와서 mapping하는 코드입니다.
    testId_mean_sum_count = load_pickle("testId_mean_sum_count.pk")
    df["testId_mean"] = df.testId.map(testId_mean_sum_count['mean'])
    df['testId_sum'] = df.testId.map(testId_mean_sum_count['sum'])

    assessmentItemID_mean_sum_count = load_pickle("assessmentItemID_mean_sum_count.pk")
    df["assessmentItemID_mean"] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['mean'])
    df['assessmentItemID_sum'] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['sum'])

    KnowledgeTag_mean_sum_count = load_pickle("KnowledgeTag_mean_sum_count.pk")
    df["tag_mean"] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['mean'])
    df['tag_sum'] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['sum'])

    print("1번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################

    ## 문제번호 (시험지 내에서) (범주형? 연속형?) - 3자리
    # 0 부터 시작하도록 
    df['problem_number'] = df.assessmentItemID.map(lambda x: int(x[-3:])-1)
    if minyoung ==True:
        df = minyong_feature(df)

    ## 시험지의(testID 별) 문제 최대개수가 몇인지 dict
    ## (이를 파악하여 중복 풀이하는 친구들을 걸러낼수있음)
    ## 0.37초 <- 아라가 짜준 코드는 1.4초
    total_num_prob_in_test = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()  
    testid_maxlen_dict = total_num_prob_in_test.to_dict()
    print("2번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 아라가 만들어준 피쳐들 추가
    if ara==True:
        df = ara_featunre(df)
    if ara_fillna_bool==True:
        df = ara_fillna(df)
    print("(3-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df.Timestamp = pd.to_datetime(df.Timestamp)

    start = time.time()  # 시작 시간 저장 
    # user별 앞 문제와 뒤 문제의 시간차이, 문제풀이에 사용한 시간
    df['problem_time_diff_in_user'] = df.groupby(['userID'])['Timestamp'].diff().shift(-1).apply(lambda x: x.total_seconds())
    df['problem_time_diff_in_user'] = df['problem_time_diff_in_user'].apply(lambda x: 300 if x>300 else x)
    # user당 마지막 row가 결측치이기 때문에 mean, median (max) 채워줌
    time_diff_fill_na = df.groupby(['userID'])['problem_time_diff_in_user'].apply(lambda x: np.mean(x.iloc[:-1]))
    fill_na_index = df[df['userID'] != df['userID'].shift(-1)].index.tolist()
    df.loc[fill_na_index, 'problem_time_diff_in_user'] = time_diff_fill_na.values
    del time_diff_fill_na, fill_na_index
    print("(3-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    start = time.time()  # 시작 시간 저장 
    df['problem_time_diff_in_user_test'] = df.groupby(['userID','testId'])['Timestamp'].diff().shift(-1).apply(lambda x: x.total_seconds())
    fill_na_index = df[df['problem_time_diff_in_user_test'].isnull()].index
    df.loc[fill_na_index,'problem_time_diff_in_user_test'] = df.groupby(['userID','testId'])['problem_time_diff_in_user_test'].mean().values
    del fill_na_index
    print("(3-3)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


    start = time.time()  # 시작 시간 저장 
    ### 진현이형 Feature
    df['time_elapsed_over_300'] = 0
    df.loc[df[df.problem_time_diff_in_user >= 300].index.tolist(), 'time_elapsed_over_300'] = 1
    if is_train==True:
        group_ass = df.groupby('assessmentItemID')
        whole_count = group_ass.size()
        correct_count = group_ass.answerCode.sum()
        wrong_count =  whole_count - correct_count
        over_count = group_ass.time_elapsed_over_300.sum()
        level_assessment_2_dict = ((correct_count - wrong_count) / (whole_count - over_count)* 100).to_dict()
        
        save_pickle('level_assessment_2_dict.pk',level_assessment_2_dict)

    level_assessment_2_dict = load_pickle('level_assessment_2_dict.pk')
    df['level_assessment'] = df['assessmentItemID'].map(level_assessment_2_dict)
    print("(3-4)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    # 시험지별 문제번호의 최대값
    # 0.877초
    max_prob_in_test = df.groupby('testId').problem_number.max()
    # (시험지별 문제개수 != 시험지별 문제번호 최댓값)에 해당하는 시험지의 index를 추출한다. (양쪽다 Series를 이용)
    inconsistent_index = max_prob_in_test[max_prob_in_test +1 != total_num_prob_in_test].index  # max_prob_in_test는 0부터 시작하고, num_prob_in_test는 1부터 시작
    # 문제 max번호와 문제개수가 일치하지 않는 시험지에 해당하는 데이터프레임으로부터
    # (시험지 Id와 문제 Id) 추출 후 문제 Id 기준 오름차순 정렬
    inconsistent_df = df.loc[df.testId.isin(inconsistent_index),['assessmentItemID', 'testId']].drop_duplicates().sort_values('assessmentItemID')

    # 순서대로 안 푼 시험지에서의 본래 문제의 순서가 dict에 저장된다 (inconsistent_Itemid_item_dict)
    inconsistent_Itemid_item_dict = {}
    inconsistent_df_group = inconsistent_df.groupby('testId')
    # 순서대로 안 푼 시험지들에 대하여 for문
    for key in inconsistent_df_group.groups:
        for i, (k,_) in enumerate(inconsistent_df_group.get_group(key).values):
            inconsistent_Itemid_item_dict[k] = i

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################

    # origin_problem_order : 한 시험지 내에서 해당 문제 Id의 본래 순서 (중간에 비어있는 문제는 없는 문제로 생각하여 다시 순서를 매김)
    # ex) A080096003,A080096001,A080096005,A080096006,A080096007,A080096008,A080096002 
    # 문제번호 : 3, 1, 5, 6, 7, 8, 2 => 문제번호 중 4번이 빠져있음
    # 따라서 4번을 제외하고 문제번호를 본래순서에 대응시키면 => (1:0,2:1,3:2,5:3,6:4,7:5,8:6)
    # 변수생성 => origin_problem_order : (2,0,3,4,5,6,1)
    # 시험 문제번호의 최대값과 시험지 내의 문제개수가 일치하지 않는 시험지의 testId들을 추출        
    df['origin_problem_order'] =  df.assessmentItemID.map(lambda x: int(inconsistent_Itemid_item_dict[x]) if x in inconsistent_Itemid_item_dict else int(x[-3:]) - 1) # 0부터 시작하도록 -1 해줌.
    print("4번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    print("5번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 유저가 푼 시험지들에 대해, 유저의 전체 정답/풀이횟수/정답률 계산
    df_group = df.groupby(['userID','testId'])['answerCode']
    print("(6-0)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_correct_cnt'] = df_group.transform(lambda x: x.cumsum().shift(1))    # 정답개수
    print("(6-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_ans_cnt'] = df_group.cumcount()  # 풀이횟수
    print("(6-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_acc'] = df['user_total_correct_cnt'] / df['user_total_ans_cnt']  # 정답률
    print("(6-3)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 유저가 풀고 있는 시험지에 대해, 현재 몇번째 문제인지
    df['total_num_prob_in_test'] = df.testId.map(testid_maxlen_dict)    # 문제수
    # 특정 시험지를 몇번째 반복하여 푸는지 계산
    df['nth_test'] = df['user_total_ans_cnt'] // df['total_num_prob_in_test'] # (같은)시험 완료 횟수
    df['user_test_ans_cnt'] = df['user_total_ans_cnt'] % df['total_num_prob_in_test']   # 현재 몇번째 문제인지
    print("7번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    # 각 시험지 당 유저의 정확도를 계산
    start = time.time()  # 시작 시간 저장 
    df['user_test_correct_cnt_per_test'] = df.groupby(['userID','testId','nth_test'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    print("(8-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    start = time.time()  # 시작 시간 저장 
    df['user_acc_per_test'] = df['user_test_correct_cnt_per_test']/df['user_test_ans_cnt']
    print("(8-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    start = time.time()  # 시작 시간 저장 
    df['int_KnowledgeTag'] = df.KnowledgeTag.astype('int64')
    df['test_total_id'] = df['assessmentItemID'].map(lambda x: int(x[1:7]))
    print("9번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    # start = time.time()  # 시작 시간 저장 
    # if jinhyun==True:
    #     df = jinhyun_feature(df)
    # print("10번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    if mjh_fillna_bool==True:
        df = mjh_fillna(df)
    return df

In [25]:
copy_df = df.copy()
copy_df['Timestamp'] = pd.to_datetime(copy_df['Timestamp'])
copy_df['problem_time_diff_in_user_test'] = copy_df.groupby(['userID','testId'])['Timestamp'].diff().shift(-1).apply(lambda x: x.total_seconds())
fill_na_index = copy_df[copy_df['problem_time_diff_in_user_test'].isnull()].index
copy_df.loc[fill_na_index,'problem_time_diff_in_user_test'] = copy_df.groupby(['userID','testId'])['problem_time_diff_in_user_test'].mean().values


In [59]:
df.groupby(['userID','testId','nth_test'])['answerCode'].transform(lambda x: x.cumsum().shift(1))

0          NaN
1          1.0
2          2.0
3          3.0
4          4.0
          ... 
2266581    1.0
2266582    NaN
2266583    1.0
2266584    2.0
2266585    3.0
Name: answerCode, Length: 2266586, dtype: float64

In [13]:

# df = feature_engineering(train, is_train=True)
# print('train feature engineering Finished!!!')
df_test = feature_engineering(test, is_train=False)
print('test feature engineering Finished!!!')


1번째 Time Lapse : 0.205857515335083
2번째 Time Lapse : 0.5822253227233887


260114it [00:03, 75363.77it/s]


(3-1)번째 Time Lapse : 5.1866068840026855
(3-2)번째 Time Lapse : 0.05714917182922363
(3-3)번째 Time Lapse : 3.2540295124053955
(3-4)번째 Time Lapse : 0.0929408073425293
4번째 Time Lapse : 0.19029569625854492
5번째 Time Lapse : 0.29859471321105957
(6-0)번째 Time Lapse : 0.0006246566772460938
(6-1)번째 Time Lapse : 19.11010503768921
(6-2)번째 Time Lapse : 0.009099245071411133
(6-3)번째 Time Lapse : 0.002373933792114258
7번째 Time Lapse : 0.038022756576538086
(8-1)번째 Time Lapse : 19.35623335838318
(8-2)번째 Time Lapse : 0.002931356430053711
9번째 Time Lapse : 0.18532133102416992
test feature engineering Finished!!!


In [15]:
interval = 10  # 구간의 간격
## time_diff 변수 구간 나누는 변수추가
bins = list(range(0, 301+interval, interval))
bins_label = [lab for lab, x in enumerate(bins)]
df["time_diff_to_bins"] = pd.cut(df["problem_time_diff_in_user"], bins, right=False, labels=bins_label[:-1])
df.time_diff_to_bins = df.time_diff_to_bins.astype(int)

df_test["time_diff_to_bins"] = pd.cut(df_test["problem_time_diff_in_user"], bins, right=False, labels=bins_label[:-1])
df_test.time_diff_to_bins = df_test.time_diff_to_bins.astype(int)

print(df.time_diff_to_bins.isnull().sum())
print(df_test.time_diff_to_bins.isnull().sum())

0
0


In [14]:
df_csv_name_after_FE = f'{upper_dir}/only_train_아라_진현_민용_41개_time_diff_bins_결측치모두채움.csv'
# df.to_csv(df_csv_name_after_FE,index=False)
df = pd.read_csv(df_csv_name_after_FE)
print(f"df_csv_name_after_FE : {df_csv_name_after_FE}")

df_csv_name_after_FE : /opt/p4-dkt-freshtomato/input/data/train_dataset/only_train_아라_진현_민용_41개_time_diff_bins_결측치모두채움.csv


In [16]:
print(f'df.shape : {df.shape}')
print(f"df_test.shape : {df_test.shape}")

df.shape : (2266586, 41)
df_test.shape : (260114, 41)


In [17]:
df_test_shift = df_test[df_test['userID'] != df_test['userID'].shift(-1)]

## 엔지니어링 이후의 EDA

In [19]:
check_df = df[df['userID'] != df['userID'].shift(-1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 마지막 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)

check_df = df_test[df_test['userID'] != df_test['userID'].shift(-1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 마지막 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)

User 별 마지막 row에 결측치가 존재하는 Column
{}
User 별 마지막 row에 결측치가 존재하는 Column
{}


In [161]:
check_df = df[df['userID'] != df['userID'].shift(1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 첫 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)

check_df = df_test[df_test['userID'] != df_test['userID'].shift(1)]
nan_check_per_column_dict = check_df.isnull().sum().to_dict()
print('User 별 첫 row에 결측치가 존재하는 Column')
over_zero_nan_per_column = {k:v for k,v in nan_check_per_column_dict.items() if v!=0}
print(over_zero_nan_per_column)


User 별 첫 row에 결측치가 존재하는 Column
{}
User 별 첫 row에 결측치가 존재하는 Column
{}


In [18]:
# 모든 데이터셋 row에 대하여 결측치가 1개 이상인 column, 결측치 개수
{k:v for k,v in df.isnull().sum().to_dict().items() if v!=0}

{}

# Permutation Importance

In [33]:
FEATS = ['answerCode','KnowledgeTag', 'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean', 'tag_sum', 'problem_number', 'test_pre', 'test_post', 'userSolTime', 'userTestTime', 'userTestAnswer', 'userTestWrong', 'userTestContAnswer', 'userTestContWrong', 'userTestCorrectRate', 'correctAnswer', 'totalAnswer', 'userAcc', 'problem_time_diff_in_user', 'time_elapsed_over_300', 'level_assessment', 'origin_problem_order', 'user_total_correct_cnt', 'user_total_ans_cnt', 'user_total_acc', 'total_num_prob_in_test', 'nth_test', 'user_test_ans_cnt', 'user_test_correct_cnt_per_test', 'user_acc_per_test', 'int_KnowledgeTag', 'test_total_id', 'time_diff_to_bins']
print(len(FEATS))
print(FEATS)

36
['answerCode', 'KnowledgeTag', 'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum', 'tag_mean', 'tag_sum', 'problem_number', 'test_pre', 'test_post', 'userSolTime', 'userTestTime', 'userTestAnswer', 'userTestWrong', 'userTestContAnswer', 'userTestContWrong', 'userTestCorrectRate', 'correctAnswer', 'totalAnswer', 'userAcc', 'problem_time_diff_in_user', 'time_elapsed_over_300', 'level_assessment', 'origin_problem_order', 'user_total_correct_cnt', 'user_total_ans_cnt', 'user_total_acc', 'total_num_prob_in_test', 'nth_test', 'user_test_ans_cnt', 'user_test_correct_cnt_per_test', 'user_acc_per_test', 'int_KnowledgeTag', 'test_total_id', 'time_diff_to_bins']


## Window Size를 고려한 Feature 추가생성 

In [ ]:
feature_with_window = ['answerCode', 'KnowledgeTag','problem_number',]

In [169]:
def add_features_with_window(df,FEATS,window_size_list):
    empty_df = pd.DataFrame(index=range(df.shape[0]))
    for window_size in tqdm_notebook(window_size_list):
        add_df = df.groupby(['userID'])[FEATS].shift(window_size).copy().add_suffix(f'_shift{window_size}')
        empty_df = pd.concat([empty_df,add_df],axis=1)

    return pd.concat([df,empty_df],axis=1)

window_size_list=list(range(1,5))  # window size : 1 ~ 4
df = add_features_with_window(df,FEATS,window_size_list)
df

  0%|          | 0/9 [00:00<?, ?it/s]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_mean,test_sum,ItemID_mean,ItemID_sum,tag_mean,tag_sum,problem_number,test_pre,test_post,diff,userSolTime,userTestTime,userTestAnswer,userTestWrong,userTestContAnswer,userTestContWrong,userTestCorrectRate,correctAnswer,totalAnswer,userAcc,problem_time_diff_in_user,time_elapsed_over_300,level_assessment,origin_problem_order,user_total_correct_cnt,user_total_ans_cnt,user_total_acc,total_num_prob_in_test,nth_test,user_test_ans_cnt,user_test_correct_cnt_per_test,user_acc_per_test,int_KnowledgeTag,test_total_id,time_diff_to_bins,answerCode_shift1,KnowledgeTag_shift1,test_mean_shift1,test_sum_shift1,ItemID_mean_shift1,ItemID_sum_shift1,tag_mean_shift1,tag_sum_shift1,problem_number_shift1,test_pre_shift1,test_post_shift1,userSolTime_shift1,userTestTime_shift1,userTestAnswer_shift1,userTestWrong_shift1,userTestContAnswer_shift1,userTestContWrong_shift1,userTestCorrectRate_shift1,correctAnswer_shift1,totalAnswer_shift1,userAcc_shift1,problem_time_diff_in_user_shift1,time_elapsed_over_300_shift1,level_assessment_shift1,origin_problem_order_shift1,user_total_correct_cnt_shift1,user_total_ans_cnt_shift1,user_total_acc_shift1,total_num_prob_in_test_shift1,nth_test_shift1,user_test_ans_cnt_shift1,user_test_correct_cnt_per_test_shift1,user_acc_per_test_shift1,int_KnowledgeTag_shift1,test_total_id_shift1,time_diff_to_bins_shift1,answerCode_shift2,KnowledgeTag_shift2,test_mean_shift2,test_sum_shift2,ItemID_mean_shift2,ItemID_sum_shift2,tag_mean_shift2,tag_sum_shift2,problem_number_shift2,test_pre_shift2,test_post_shift2,userSolTime_shift2,userTestTime_shift2,userTestAnswer_shift2,userTestWrong_shift2,userTestContAnswer_shift2,userTestContWrong_shift2,userTestCorrectRate_shift2,correctAnswer_shift2,totalAnswer_shift2,userAcc_shift2,problem_time_diff_in_user_shift2,time_elapsed_over_300_shift2,level_assessment_shift2,origin_problem_order_shift2,user_total_correct_cnt_shift2,user_total_ans_cnt_shift2,user_total_acc_shift2,total_num_prob_in_test_shift2,nth_test_shift2,user_test_ans_cnt_shift2,user_test_correct_cnt_per_test_shift2,user_acc_per_test_shift2,int_KnowledgeTag_shift2,test_total_id_shift2,time_diff_to_bins_shift2,answerCode_shift3,KnowledgeTag_shift3,test_mean_shift3,test_sum_shift3,ItemID_mean_shift3,ItemID_sum_shift3,tag_mean_shift3,tag_sum_shift3,problem_number_shift3,test_pre_shift3,test_post_shift3,userSolTime_shift3,userTestTime_shift3,userTestAnswer_shift3,userTestWrong_shift3,userTestContAnswer_shift3,userTestContWrong_shift3,userTestCorrectRate_shift3,correctAnswer_shift3,totalAnswer_shift3,userAcc_shift3,problem_time_diff_in_user_shift3,time_elapsed_over_300_shift3,level_assessment_shift3,origin_problem_order_shift3,user_total_correct_cnt_shift3,user_total_ans_cnt_shift3,user_total_acc_shift3,total_num_prob_in_test_shift3,nth_test_shift3,user_test_ans_cnt_shift3,user_test_correct_cnt_per_test_shift3,user_acc_per_test_shift3,int_KnowledgeTag_shift3,test_total_id_shift3,time_diff_to_bins_shift3,answerCode_shift4,KnowledgeTag_shift4,test_mean_shift4,test_sum_shift4,ItemID_mean_shift4,ItemID_sum_shift4,tag_mean_shift4,tag_sum_shift4,problem_number_shift4,test_pre_shift4,test_post_shift4,userSolTime_shift4,userTestTime_shift4,userTestAnswer_shift4,userTestWrong_shift4,userTestContAnswer_shift4,userTestContWrong_shift4,userTestCorrectRate_shift4,correctAnswer_shift4,totalAnswer_shift4,userAcc_shift4,problem_time_diff_in_user_shift4,time_elapsed_over_300_shift4,level_assessment_shift4,origin_problem_order_shift4,user_total_correct_cnt_shift4,user_total_ans_cnt_shift4,user_total_acc_shift4,total_num_prob_in_test_shift4,nth_test_shift4,user_test_ans_cnt_shift4,user_test_correct_cnt_per_test_shift4,user_acc_per_test_shift4,int_KnowledgeTag_shift4,test_total_id_shift4,time_diff_to_bins_shift4,answerCode_shift5,KnowledgeTag_shift5,test_mean_shift5,test_sum_shift5,ItemID_mean_shift5,ItemID_sum_shift5,tag_mean_shift5,tag_sum_shift5,problem_number_shift5,test_pre_shift5,tes

In [171]:
print(df.shape)
df.dropna(axis=0,inplace=True)
print(df.shape)


(2266586, 365)


MemoryError: Unable to allocate 5.56 GiB for an array with shape (338, 2206304) and data type float64

## Valid Score 측정을 위한 Train-Val SPlit 과정

In [117]:
correct_rate_per_user = df.groupby(['userID'])['answerCode'].mean()
correct_rate_per_user.min()
# df['answerCode'][last_user_idxes].astype(str)

0.0

In [77]:
assessmentItemID_in_test_set = set(df_test_shift.assessmentItemID)
testId_in_test_set = set(df_test_shift.testId)
tag_in_test_set = set(df_test_shift.KnowledgeTag)

print(df.loc[df.testId.isin(testId_in_test_set)]['answerCode'].mean())
print(df.loc[df.assessmentItemID.isin(assessmentItemID_in_test_set)]['answerCode'].mean())
print(df.loc[df.KnowledgeTag.isin(tag_in_test_set)]['answerCode'].mean())


0.6554042066988823
0.5593832204623048
0.6673991267106635


In [81]:
df.groupby('userID')['answerCode'].mean().mean()

0.6289091262766637

In [82]:
df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_mean,test_sum,ItemID_mean,ItemID_sum,tag_mean,tag_sum,problem_number,test_pre,test_post,diff,userSolTime,userTestTime,userTestAnswer,userTestWrong,userTestContAnswer,userTestContWrong,userTestCorrectRate,correctAnswer,totalAnswer,userAcc,problem_time_diff_in_user,time_elapsed_over_300,level_assessment,origin_problem_order,user_total_correct_cnt,user_total_ans_cnt,user_total_acc,total_num_prob_in_test,nth_test,user_test_ans_cnt,user_test_correct_cnt_per_test,user_acc_per_test,int_KnowledgeTag,test_total_id,time_diff_to_bins
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.947683,1268,0.982063,219,0.955022,637,0,60,1,0 days 00:00:03,3.00,3.0,0,0,0,0,0.000000,0.0,0,0.0,3.0,0,96.846847,0,0.0,0,0.0,6,0,0,0.0,0.0,7224,60001,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0.947683,1268,0.964126,215,0.913187,3040,1,60,1,0 days 00:00:08,5.50,11.0,1,0,1,0,0.500000,1.0,1,1.0,8.0,0,94.090909,1,1.0,1,1.0,6,0,1,1.0,1.0,7225,60001,0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0.947683,1268,0.910314,203,0.913187,3040,2,60,1,0 days 00:00:07,6.00,18.0,2,0,2,0,0.666667,2.0,2,1.0,7.0,0,82.062780,2,2.0,2,1.0,6,0,2,2.0,1.0,7225,60001,0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0.947683,1268,0.968610,216,0.913187,3040,3,60,1,0 days 00:00:07,6.25,25.0,3,0,3,0,0.750000,3.0,3,1.0,7.0,0,94.144144,3,3.0,3,1.0,6,0,3,3.0,1.0,7225,60001,0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0.947683,1268,0.941704,210,0.913187,3040,4,60,1,0 days 00:00:11,7.20,36.0,4,0,4,0,0.800000,4.0,4,1.0,11.0,0,89.140271,4,4.0,4,1.0,6,0,4,4.0,1.0,7225,60001,1


In [52]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
#LGBMC = lgb.LGBMClassifier(n_estimators=100, n_jobs=-1)
#LGBMC.fit(x_train, y_train)
train = df[FEATS]
x = train.drop(['answerCode'],axis=1)
y = train['answerCode']

unique_user_id = list(set(df.userID))
last_user_idxes = df[df.userID!=df.userID.shift(-1)].index

# stratify_key = df['answerCode'][last_user_idxes]
stratify_key = df.groupby(['userID'])['answerCode'].mean().apply(lambda x: 1 if x>0.610425595902904 else 0)



tr_idx, val_idx = train_test_split(unique_user_id,  train_size=0.7, stratify=stratify_key, random_state=42)
tr_idx = df.loc[df.userID.isin(tr_idx)].index
val_idx = df.loc[df.userID.isin(val_idx)].index

print(len(tr_idx))
print(len(val_idx))
not_last_user_idx = set(val_idx) - set(last_user_idxes)
val_idx = list(set(val_idx)-not_last_user_idx)
tr_idx = list(set(tr_idx)|not_last_user_idx)

print(len(tr_idx))
print(len(val_idx))

tr_X, val_X = x.iloc[tr_idx], x.iloc[val_idx]
tr_y, val_y = y.iloc[tr_idx], y.iloc[val_idx]

print(tr_X.shape)
print(val_X.shape)
print(tr_y.shape)
print(val_y.shape)
print(f'train의 정답률 분포 : {tr_y.mean()}')
print(f'valid의 정답률 분포 : {val_y.mean()}')

1581800
684786
2264576
2010
(2264576, 35)
(2010, 35)
(2264576,)
(2010,)


In [53]:

cat_features = ['test_pre','time_elapsed_over_300']

params = {
    'objective': 'binary',
    'max_depth': -1,  # 18,20,-1,30,25,15,13, 11,12,10
    'learning_rate': 0.1, 
    "boosting": "gbdt",#'dart','goss',
    "metric": "auc",
    "verbosity": -1,
    'seed':42,
}
d_train = lgb.Dataset(tr_X, label=tr_y, categorical_feature=cat_features)
d_valid = lgb.Dataset(val_X, label=val_y, categorical_feature=cat_features)
watchlist = [d_train, d_valid]

print('training LGB:')
model = lgb.train(params,
                    train_set=d_train,
                    num_boost_round=10000,
                    valid_sets=watchlist,
                    verbose_eval=200,
                    early_stopping_rounds=750,
                    )

training LGB:
Training until validation scores don't improve for 750 rounds
[200]	training's auc: 0.850571	valid_1's auc: 0.822629
[400]	training's auc: 0.852716	valid_1's auc: 0.823128
[600]	training's auc: 0.854562	valid_1's auc: 0.823344
[800]	training's auc: 0.856238	valid_1's auc: 0.823163
[1000]	training's auc: 0.857878	valid_1's auc: 0.823015
[1200]	training's auc: 0.859307	valid_1's auc: 0.823509
[1400]	training's auc: 0.860769	valid_1's auc: 0.822979
[1600]	training's auc: 0.862082	valid_1's auc: 0.823204
[1800]	training's auc: 0.863367	valid_1's auc: 0.822864
Early stopping, best iteration is:
[1194]	training's auc: 0.859272	valid_1's auc: 0.823669


train의 정답률 분포 : 0.654534005482704
valid의 정답률 분포 : 0.4791044776119403


In [ ]:
# import eli5 
# from eli5.sklearn import PermutationImportance
# perm = PermutationImportance(model, scoring = "auc", random_state = 42).fit(val_X, val_y)
# eli5.show_weights(perm, top = 80, feature_names = val_X.columns.tolist())


In [ ]:
# Customizing Permutation Importance
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from collections import defaultdict
import warnings
warnings.filterwarnings(action='ignore')

metric = roc_auc_score
seed_list = [42,77,120,200,350,565,777,853,952,1000,1997] # Various Seeds
threshold = 1e-4

def permutation_importance(model, val_X, val_y, metric, threshold,
                           minimize=False, verbose=True,seed=seed_list):
    result_dict = defaultdict(list)
    n_belong_bad_feats = defaultdict(int)

    y_pred = model.predict(val_X, num_iteration=model.best_iteration,cat_features=cat_features)
    result_dict['base_score'] = metric(val_y, y_pred)
    if verbose:
        print(f"base score : {result_dict['base_score']}")

    for seed in tqdm_notebook(seed_list):
        np.random.seed(seed)
        random.seed(seed)
        for col in tqdm_notebook(val_X.columns):
            freezed_col = val_X[col].copy()
            val_X[col] = np.random.permutation(val_X[col])
            preds = model.predict(val_X, num_iteration=model.best_iteration,cat_features=cat_features)
            score_after = metric(val_y, preds)
            result_dict[col].append(score_after)
            if minimize:
                if score_after<=result_dict['base_score']-threshold:
                    n_belong_bad_feats[col]+=1
            else:
                if score_after>=result_dict['base_score']+threshold:
                    n_belong_bad_feats[col]+=1
            val_X[col] = freezed_col

    return result_dict, n_belong_bad_feats

result_dict, n_belong_bad_feats = permutation_importance(model, val_X, val_y, metric, threshold=threshold,
                           minimize=False, verbose=True,seed=seed_list)

# base score : 0.8114367212457887

In [102]:
bad_feats_over_half = {column_name:n_count for column_name, n_count in n_belong_bad_feats.items() if n_count>=len(seed_list)//2}
bad_feats_over_half

{'problem_number': 8,
 'test_pre': 9,
 'test_post': 11,
 'totalAnswer': 9,
 'total_num_prob_in_test': 6}

In [99]:
threshold = 1e-4
mean_score_dict = {k:np.mean(v) for k,v in result_dict.items()}

bad_features_by_mean = []

for column_name, mean_score in mean_score_dict.items():
    if mean_score>=result_dict['base_score']+threshold:
        bad_features_by_mean.append(column_name)

bad_features_by_mean

['problem_number',
 'test_pre',
 'test_post',
 'totalAnswer',
 'total_num_prob_in_test']

In [71]:
pd.DataFrame(results.values())

,0,1
0,base_score,0.823669
1,KnowledgeTag,0.823313
2,test_mean,0.811111
3,test_sum,0.822671
4,ItemID_mean,0.669523
5,ItemID_sum,0.821329
6,tag_mean,0.821637
7,tag_sum,0.823441
8,problem_number,0.823716
9,test_pre,0.821412
